# BASE IMPORTS

In [81]:
# !pip install tensorflow

In [82]:
import sys, danbi as bi
sys.path.append("../..")
import _lib_ as lib

import pandas_ta as ta
from danbi.extends import bibokeh as bibo

bibo.setJupyterEnable()


from utils.converters import convert_to_df, convert_to_df_float
import pandas as pd

import glob

from IPython.display import clear_output
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout



Loading BokehJS ...

In [10]:
import json
from collections import OrderedDict
import pandas as pd


class OrderBookCSVGenerator:
    def __init__(self, initial_snapshot, partial_depth_stream_data, max_levels=10, slice_size=100000):
        self.slice_size = slice_size
        self.max_levels = max_levels
        self.bids = []#pd.DataFrame(columns=['price', 'volume'], dtype=float)
        self.asks = []#pd.DataFrame(columns=['price', 'volume'], dtype=float)
        self.timestamps = []
        self.train_data = {}


        try:
            self.orderbook = self.initialize_orderbook(initial_snapshot)
            self.add_train_data()
            

        except Exception as e:
            print(f"Error initializing order book: {e}")

        try:
            for i,message in enumerate(partial_depth_stream_data):
                if i%50==0:
                    clear_output(wait=True)
                if i%self.slice_size==0:
                    self.snapshot_to_csv()

                print("Data slice saved...{}".format(len(self.train_data)))
                # print("Data slice shape...{}".format(self.train_data[-1].shape))


                self.update_book(message)
                # self.add_train_data()

        except Exception as e:
            print(f"Error updating order book: {e}")
            
        # try:
        #     for i,df_slice in enumerate(self.train_data):
        #         df_slice.to_csv(f"data_{i+1}.csv", index=False)

        except Exception as e:
            print(f"Error slicing order book: {e}")
            # for i,df_slice in enumerate(self.train_data):
            #     df_slice.to_csv(f"error_recovery_data_{i+1}.csv", index=False)
            
        

    def add_train_data(self):
        cur_orderbook_timestamp=self.orderbook['message_output_time']
        # print(self.bids, self.asks, self.timestamps)
        recent_levels = {'bids':[(k,v) for k,v in list(self.orderbook['bids'].items())[:self.max_levels]],
                         'asks': [(k,v) for k,v in list(self.orderbook['asks'].items())[:self.max_levels]]}
        row_data={}


    #     for i in range(self.max_levels):

        for i in range(self.max_levels):
            bid=recent_levels['bids'][i]
            ask=recent_levels['asks'][i]

            row_data[f'bid_price_{i+1}']=bid[0]
            row_data[f'bid_volume_{i+1}']=bid[1]
            row_data[f'ask_price_{i+1}']=ask[0]
            row_data[f'ask_volume_{i+1}']=ask[1]
            # row_data.append(bid[0])
            # row_data.append(bid[1])
            # row_data.append(ask[0])
            # row_data.append(ask[1])
        
        self.train_data[str(cur_orderbook_timestamp)]= row_data
        self.timestamps.append(cur_orderbook_timestamp)
        
        print("cur_orderbook_timestamp", cur_orderbook_timestamp)
        print("self.timestamps", self.timestamps[-1])

        # # self.timestamps.append(cur_orderbook_timestamp)
        if len(self.timestamps)%1000==0:
            print("Add train data successful.")
            print("Timestamps length", len(self.timestamps))
            # self.timestamps = []
            # self.bids = []
            # self.asks = []


    def initialize_orderbook(self, initial_snapshot):
        """
        Parse the initial snapshot and populate the order book.
        """
        data = json.loads(initial_snapshot)
        bids={}
        asks={}
        try:
            last_update_id=data['lastUpdateId']
            message_output_time=data['E']
            transaction_time=data['T']
            
        except:
            raise ValueError("The orderbook stream data is not in the expected format.")
        
        # bids = convert_to_df(bids, ['price', 'quantity']).sort_values(by='price', ascending=False, inplace=True)
        # asks = convert_to_df(asks, ['price', 'quantity']).sort_values(by='price', ascending=True, inplace=True)
        

        # Fill initial bids and asks
        for bid in data['bids']:
            price, quantity = bid
            bids[price] = quantity
        
        for ask in data['asks']:
            price, quantity = ask
            asks[price] = quantity

    
        orderbook = {
            'last_update_id': last_update_id,
            'message_output_time': message_output_time,
            'update_time': transaction_time,
            "bids": OrderedDict(sorted(bids.items(), key=lambda x: x[0], reverse=True)),
            "asks": OrderedDict(sorted(asks.items(), key=lambda x: x[0]))
        }
        
        
        print("Order book initialized.")
        return orderbook
        
    
    def update_book(self, message):
        data = json.loads(message)
        message_type = data['e']
        if message_type == 'depthUpdate':
            if len(self.timestamps)<2:
                self.apply_first_update(data)
            else:
                self.apply_update(data)   
        else:
            print(f"Message type {message_type} is not supported.")
    
    def apply_first_update(self, data):
        """
        Apply the first depth update to the order book.
        """
        print("Applying first update...")
        # if data['U'] > self.orderbook['last_update_id']:
        #     print("Update data may have skipped some updates. Still proceeding with the update.")
        if data['u'] < self.orderbook['last_update_id']:
            print("Update data is older than the current order book.")
            return self.orderbook
            # raise ValueError("Order book update ids are not sequential.")  
        self.orderbook['last_update_id'] = data['u']
        self.orderbook['message_output_time'] = data['E']
        self.orderbook['update_time'] = data['T']

        
        # Update bids
        for bid in data['b']:
            price, quantity = bid
            if float(quantity) == 0:
                # Remove bid if quantity is 0
                if price in self.orderbook['bids']:
                    del self.orderbook['bids'][price]
            else:
                # Update the bid price
                self.orderbook['bids'][price] = quantity
        
        # Update asks
        for ask in data['a']:
            price, quantity = ask
            if float(quantity) == 0:
                # Remove ask if quantity is 0
                if price in self.orderbook['asks']:
                    del self.orderbook['asks'][price]
            else:
                # Update the ask price
                self.orderbook['asks'][price]
        
        self.add_train_data()
        print("First update applied.")
        print()


    def apply_update(self, data):
        """
        Apply depth update to the order book.
        """
        try:
            # print(data)
            U=data['U']
            u=data['u']
            last_update_id=self.orderbook['last_update_id']

            if U>last_update_id:
                print("Update data may have skipped some updates. Still proceeding with the update.")


            if u < last_update_id:
                print("Update data is older than the current order book.")
                return self.orderbook
                # raise ValueError("Order book update ids are not sequential.")
            
            
            
            self.orderbook['last_update_id'] = u
            self.orderbook['update_time'] = data['E']
            self.orderbook['message_output_time'] = data['T']

            # Update bids
            for bid in data['b']:
                price, quantity = bid
                if float(quantity) == 0:
                    # Remove bid if quantity is 0
                    if price in self.orderbook['bids']:
                        del self.orderbook['bids'][price]
                else:
                    # Update the bid price
                    self.orderbook['bids'][price] = quantity
            
            # Update asks
            for ask in data['a']:
                price, quantity = ask
                if float(quantity) == 0:
                    # Remove ask if quantity is 0
                    if price in self.orderbook['asks']:
                        del self.orderbook['asks'][price]
                else:   
                    # Update the ask price
                    self.orderbook['asks'][price] = quantity
        except Exception as e:
            print(f"Error applying update: {e}")
            print('Event time:', data['E'])

        
        # print("Update applied.")
        self.add_train_data()
    # def snapshot_to_csv(self):
    #     df = pd.DataFrame()
    #     df.index = pd.to_datetime(self.timestamps)

    #     for i in range(self.max_levels):
    #         bid_price_col = f'bid_price_{i+1}'
    #         bid_volume_col = f'bid_volume_{i+1}'
    #         ask_price_col = f'ask_price_{i+1}'
    #         ask_volume_col = f'ask_volume_{i+1}'

    #         df[bid_price_col] = [list(bid.keys())[i] if i < len(bid) else None for bid in self.bids]
    #         df[bid_volume_col] = [list(bid.values())[i] if i < len(bid) else None for bid in self.bids]
    #         df[ask_price_col] = [list(ask.keys())[i] if i < len(ask) else None for ask in self.asks]
    #         df[ask_volume_col] = [list(ask.values())[i] if i < len(ask) else None for ask in self.asks]

    #     return df
    def snapshot_to_csv(self):
        # print(self.train_data)
        df = pd.DataFrame.from_dict(self.train_data, orient='index')
        df=convert_to_df(df)
        print(df.head())
        df.to_csv(f"data_{len(self.train_data)}.csv", index=True)
        
        return df


In [11]:
# file_names = glob.glob('data/*')
file_names = glob.glob('data/ape_futuresDepth_4.json')
file_names

['data/ape_futuresDepth_4.json']

In [12]:
with open('data/ape_futuresDepth_4.json','r') as f:
    messages = [line.strip() for line in f.readlines() if line.strip()!='']


In [13]:

initial_snapshot=messages[1]
partial_depth_stream_data=messages[2:]

In [15]:
ob = OrderBookCSVGenerator(initial_snapshot, partial_depth_stream_data, max_levels=10)

Data slice saved...983388
Update data may have skipped some updates. Still proceeding with the update.
cur_orderbook_timestamp 1730885338553
self.timestamps 1730885338553
Data slice saved...983389
Update data may have skipped some updates. Still proceeding with the update.
cur_orderbook_timestamp 1730885338681
self.timestamps 1730885338681
Data slice saved...983390
Update data may have skipped some updates. Still proceeding with the update.
cur_orderbook_timestamp 1730885338795
self.timestamps 1730885338795
Data slice saved...983391
Update data may have skipped some updates. Still proceeding with the update.
cur_orderbook_timestamp 1730885338924
self.timestamps 1730885338924
Data slice saved...983392
Update data may have skipped some updates. Still proceeding with the update.
cur_orderbook_timestamp 1730885339037
self.timestamps 1730885339037
Data slice saved...983393
Update data may have skipped some updates. Still proceeding with the update.
cur_orderbook_timestamp 1730885339150
self

In [62]:
import pandas as pd

df=pd.DataFrame.from_dict(ob.train_data, orient='index')

In [63]:
df.columns

Index(['bid_price_1', 'bid_volume_1', 'ask_price_1', 'ask_volume_1',
       'bid_price_2', 'bid_volume_2', 'ask_price_2', 'ask_volume_2',
       'bid_price_3', 'bid_volume_3', 'ask_price_3', 'ask_volume_3',
       'bid_price_4', 'bid_volume_4', 'ask_price_4', 'ask_volume_4',
       'bid_price_5', 'bid_volume_5', 'ask_price_5', 'ask_volume_5',
       'bid_price_6', 'bid_volume_6', 'ask_price_6', 'ask_volume_6',
       'bid_price_7', 'bid_volume_7', 'ask_price_7', 'ask_volume_7',
       'bid_price_8', 'bid_volume_8', 'ask_price_8', 'ask_volume_8',
       'bid_price_9', 'bid_volume_9', 'ask_price_9', 'ask_volume_9',
       'bid_price_10', 'bid_volume_10', 'ask_price_10', 'ask_volume_10'],
      dtype='object')

In [64]:

df.head()

,bid_price_1,bid_volume_1,ask_price_1,ask_volume_1,bid_price_2,bid_volume_2,ask_price_2,ask_volume_2,bid_price_3,bid_volume_3,...,ask_price_8,ask_volume_8,bid_price_9,bid_volume_9,ask_price_9,ask_volume_9,bid_price_10,bid_volume_10,ask_price_10,ask_volume_10
1730778033102,0.9142,1085,0.9143,731,0.9141,3278,0.9144,3063,0.9140,2713,...,0.9150,6491,0.9134,23457,0.9151,4775,0.9133,21333,0.9152,13008
1730778033146,0.9142,17,0.9143,731,0.9141,3038,0.9144,3063,0.9140,2713,...,0.9150,6491,0.9134,23457,0.9151,4775,0.9133,21333,0.9152,13008
1730778033221,0.9142,17,0.9143,750,0.9141,3038,0.9144,3429,0.9140,2407,...,0.9150,6491,0.9134,23457,0.9151,4775,0.9133,21333,0.9152,13008
1730778033366,0.9142,2438,0.9144,2099,0.9141,3038,0.9145,5154,0.9140,2887,...,0.9151,4775,0.9134,23457,0.9152,13336,0.9133,21333,0.9153,10962
1730778033473,0.9142,2689,0.9144,2131,0.9141,3289,0.9145,4444,0.9140,4277,...,0.9151,4775,0.9134,23457,0.9152,14320,0.9133,21326,0.9153,10962


In [83]:
df=convert_to_df_float(df)
df['timestamp']=df.index.values.tolist()
df['timestamp']=df['timestamp'].apply(lambda x: int(x))



In [66]:
df.columns

Index(['bid_price_1', 'bid_volume_1', 'ask_price_1', 'ask_volume_1',
       'bid_price_2', 'bid_volume_2', 'ask_price_2', 'ask_volume_2',
       'bid_price_3', 'bid_volume_3', 'ask_price_3', 'ask_volume_3',
       'bid_price_4', 'bid_volume_4', 'ask_price_4', 'ask_volume_4',
       'bid_price_5', 'bid_volume_5', 'ask_price_5', 'ask_volume_5',
       'bid_price_6', 'bid_volume_6', 'ask_price_6', 'ask_volume_6',
       'bid_price_7', 'bid_volume_7', 'ask_price_7', 'ask_volume_7',
       'bid_price_8', 'bid_volume_8', 'ask_price_8', 'ask_volume_8',
       'bid_price_9', 'bid_volume_9', 'ask_price_9', 'ask_volume_9',
       'bid_price_10', 'bid_volume_10', 'ask_price_10', 'ask_volume_10',
       'timestamp'],
      dtype='object')

In [67]:
df=df[['timestamp','bid_price_1', 'bid_volume_1', 'ask_price_1', 'ask_volume_1',
       'bid_price_2', 'bid_volume_2', 'ask_price_2', 'ask_volume_2',
       'bid_price_3', 'bid_volume_3', 'ask_price_3', 'ask_volume_3',
       'bid_price_4', 'bid_volume_4', 'ask_price_4', 'ask_volume_4',
       'bid_price_5', 'bid_volume_5', 'ask_price_5', 'ask_volume_5',
       'bid_price_6', 'bid_volume_6', 'ask_price_6', 'ask_volume_6',
       'bid_price_7', 'bid_volume_7', 'ask_price_7', 'ask_volume_7',
       'bid_price_8', 'bid_volume_8', 'ask_price_8', 'ask_volume_8',
       'bid_price_9', 'bid_volume_9', 'ask_price_9', 'ask_volume_9',
       'bid_price_10', 'bid_volume_10', 'ask_price_10', 'ask_volume_10'
       ]]

In [84]:
df

,timestamp,bid_price_1,bid_volume_1,ask_price_1,ask_volume_1,bid_price_2,bid_volume_2,ask_price_2,ask_volume_2,bid_price_3,...,ask_price_8,ask_volume_8,bid_price_9,bid_volume_9,ask_price_9,ask_volume_9,bid_price_10,bid_volume_10,ask_price_10,ask_volume_10
1730778033102,1730778033102,0.9142,1085.0,0.9143,731.0,0.9141,3278.0,0.9144,3063.0,0.9140,...,0.9150,6491.0,0.9134,23457.0,0.9151,4775.0,0.9133,21333.0,0.9152,13008.0
1730778033146,1730778033146,0.9142,17.0,0.9143,731.0,0.9141,3038.0,0.9144,3063.0,0.9140,...,0.9150,6491.0,0.9134,23457.0,0.9151,4775.0,0.9133,21333.0,0.9152,13008.0
1730778033221,1730778033221,0.9142,17.0,0.9143,750.0,0.9141,3038.0,0.9144,3429.0,0.9140,...,0.9150,6491.0,0.9134,23457.0,0.9151,4775.0,0.9133,21333.0,0.9152,13008.0
1730778033366,1730778033366,0.9142,2438.0,0.9144,2099.0,0.9141,3038.0,0.9145,5154.0,0.9140,...,0.9151,4775.0,0.9134,23457.0,0.9152,13336.0,0.9133,21333.0,0.9153,10962.0
1730778033473,1730778033473,0.9142,2689.0,0.9144,2131.0,0.9141,3289.0,0.9145,4444.0,0.9140,...,0.9151,4775.0,0.9134,23457.0,0.9152,14320.0,0.9133,21326.0,0.9153,10962.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1730885339037,1730885339037,0.9119,246.0,1.0550,664.0,0.9117,5188.0,1.0551,28.0,0.9116,...,1.0564,35.0,0.9110,86.0,1.0557,48.0,0.9109,16.0,1.0558,7.0
1730885339150,1730885339150,0.9119,246.0,1.0550,664.0,0.9117,5188.0,1.0551,28.0,0.9116,...,1.0564,35.0,0.9110,86.0,1.0557,48.0,0.9109,16.0,1.0558,7.0
1730885339242,1730885339242,0.9119,246.0,1.0550,664.0,0.9117,5188.0,1.0551,28.0,0.9116,...,1.0564,35.0,0.9110,86.0,1.0557,48.0,0.9109,16.0,1.0558,7.0
1730885339359,1730885339359,0.9119,246.0,1.0550,664.0,0.9117,5188.0,1.0551,28.0,0.9116,...,1.0564,35.0,0.9110,86.0,1.0557,48.0,0.9109,16.0,1.0558,7.0


In [85]:
df.dtypes

timestamp          int64
bid_price_1      float64
bid_volume_1     float64
ask_price_1      float64
ask_volume_1     float64
bid_price_2      float64
bid_volume_2     float64
ask_price_2      float64
ask_volume_2     float64
bid_price_3      float64
bid_volume_3     float64
ask_price_3      float64
ask_volume_3     float64
bid_price_4      float64
bid_volume_4     float64
ask_price_4      float64
ask_volume_4     float64
bid_price_5      float64
bid_volume_5     float64
ask_price_5      float64
ask_volume_5     float64
bid_price_6      float64
bid_volume_6     float64
ask_price_6      float64
ask_volume_6     float64
bid_price_7      float64
bid_volume_7     float64
ask_price_7      float64
ask_volume_7     float64
bid_price_8      float64
bid_volume_8     float64
ask_price_8      float64
ask_volume_8     float64
bid_price_9      float64
bid_volume_9     float64
ask_price_9      float64
ask_volume_9     float64
bid_price_10     float64
bid_volume_10    float64
ask_price_10     float64


In [86]:
df_test=df[df.index>"1730865643033"].copy()

In [87]:
df_test.shape

(183409, 41)

In [88]:
df_train=df[df.index<="1730865643033"].copy()

In [89]:
df_train.shape

(799988, 41)

In [90]:
df_train.to_csv("data_train.csv",index=False, header=False, encoding='utf-8')
df_test.to_csv("data_test.csv",index=False, header=False, encoding='utf-8')

In [76]:


# Open the file in binary mode
with open('data_train.csv', 'rb') as file:
    # Read the file's content in binary
    content = file.read()

# Print the raw bytes around the problematic position (example, bytes 3100-3150)
start_byte = 3100
end_byte = 3150
print(content[start_byte:end_byte])

b'8614,0.9133,21874,0.9155,6846\n1730778034416,0.9142'
